In [ ]:
import matplotlib
#matplotlib.use('TkAgg')
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import re
from scipy import stats
import seaborn as sns
from scipy.stats import spearmanr
from scipy.stats import ttest_ind
import matplotlib as mpl
import plotly.graph_objects as go
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import mannwhitneyu
from matplotlib.colors import ListedColormap, BoundaryNorm


一些需要单独运行的函数：
cal_attended_index() (分别用于四维的和三维的数据，有两个)
food_preference_arr()
get_dim_variance()
plot_3d_trace()
plot_3d_trace_score()
dim_probed()
probe_segments_length()
dim_probe_counts()
probe_sequence（）
probe_fraction（）
stick_fraction（）
stick_sequence（）
extract_key_From_filename（）
get_round_behavour（）
is_testing_trial（）
get_stick_trial（）


In [4]:
#for p2 p2only (12个食物)
def cal_attended_index(round):
    # a1 = 0, a2 = 0, a3 = 0
    # b1 = 0, b2 = 0, b3 = 0
    # c1 = 0, c2 = 0, c3 = 0
    # d1 = 0, d2 = 0, d3 = 0
    block1_list = [0]*12
    block2_list = [0]*12
    block3_list = [0]*12
    
    for choice in round[0]:
        choice_str = str(choice)
        i = 0
        for i in range(len(choice_str)):
            if choice_str[i] == '1':
                block1_list[0+i*3] +=1
            if choice_str[i] == '2' :
                block1_list[1+i*3] +=1
            if choice_str[i] == '3':
                block1_list[2+i*3] +=1
            i+=1
    for choice in round[1]:
        choice_str = str(choice)
        i = 0
        for i in range(len(choice_str)):
            if choice_str[i] == '1':
                block2_list[0+i*3] +=1
            if choice_str[i] == '2' :
                block2_list[1+i*3] +=1
            if choice_str[i] == '3':
                block2_list[2+i*3] +=1
            i+=1
    for choice in round[2]:
        choice_str = str(choice)
        i = 0
        for i in range(len(choice_str)):
            if choice_str[i] == '1':
                block3_list[0+i*3] +=1
            if choice_str[i] == '2' :
                block3_list[1+i*3] +=1
            if choice_str[i] == '3':
                block3_list[2+i*3] +=1
            i+=1
    weights = [27,15,3]
    food_arr = np.array([block1_list,block2_list,block3_list])
    food_index = np.dot(weights, food_arr)
    return food_index

In [ ]:
#for p1 (9个食物) (跑p1数据前要重跑这个函数)
def cal_attended_index(round):
    block1_list = [0]*9
    block2_list = [0]*9
    block3_list = [0]*9
    
    for choice in round[0]:
        choice_str = str(choice)
        i = 0
        for i in range(len(choice_str)):
            if choice_str[i] == '1':
                block1_list[0+i*3] +=1
            if choice_str[i] == '2' :
                block1_list[1+i*3] +=1
            if choice_str[i] == '3':
                block1_list[2+i*3] +=1
            i+=1
    for choice in round[1]:
        choice_str = str(choice)
        i = 0
        for i in range(len(choice_str)):
            if choice_str[i] == '1':
                block2_list[0+i*3] +=1
            if choice_str[i] == '2' :
                block2_list[1+i*3] +=1
            if choice_str[i] == '3':
                block2_list[2+i*3] +=1
            i+=1
    for choice in round[2]:
        choice_str = str(choice)
        i = 0
        for i in range(len(choice_str)):
            if choice_str[i] == '1':
                block3_list[0+i*3] +=1
            if choice_str[i] == '2' :
                block3_list[1+i*3] +=1
            if choice_str[i] == '3':
                block3_list[2+i*3] +=1
            i+=1
    weights = [27,9,3]
    food_arr = np.array([block1_list,block2_list,block3_list])
    food_index = np.dot(weights, food_arr)
    return food_index

In [ ]:
def food_preference_arr(choice_sequence):
    food_index_arr_list = []
    data_int = choice_sequence.astype(int)
    for round_idx, round in enumerate(data_int):
        food_index_arr_list.append(cal_attended_index(round))
    food_index_arr = np.array(food_index_arr_list)
    return food_index_arr

In [ ]:
#定义从food index计算dim variance的函数：----- for p1  
def get_dim_variance(food_index_array):
    #输入是每个被试的food_index序列on trial course (n * 12)
    #输出应该是每个被试的dim_variance序列on trial course (n * 4 )
    variance_list = []
    for round in food_index_array:
        variances = [np.var(round[i:i+3]) for i in range(0, len(round), 3)]
        variance_list.append(variances)
    variance_sequence_arr = np.array(variance_list)
    #进行归一化：
    min_values = variance_sequence_arr.min()
    max_values = variance_sequence_arr.max()
    normalized_var_seq_arr = (variance_sequence_arr - min_values) / (max_values - min_values)
    return normalized_var_seq_arr




In [ ]:
#定义3D可视化的函数：(on trial course)
def plot_3d_trace(df, array_column_name, subject_column_name,score_column_name, save_dir):
    import os
    from matplotlib.lines import Line2D
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for index, row in df.iterrows():
        # 获取Subject值和对应的数组
        subject_value = row[subject_column_name]
        array_data = row[array_column_name]
        score_value = np.max(row[score_column_name])

        # 创建一个3D图形
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d') #降维到3维时改为3，降维到2维时改为2

        for i in range(len(array_data)):
            # 计算前后两个点之间的距离，如果距离较远，则使用不同的颜色
            color_value = ( i / len(array_data))  # 归一化时间索引
            if i == 0:
                ax.plot(array_data[i, 0], array_data[i, 1],array_data[i, 2], 'bo', markersize=13, label='Start Point') #
            elif i == len(array_data)-1:
                ax.plot(array_data[i, 0], array_data[i, 1],array_data[i, 2], color='orange', marker='o', markersize=13, label='End Point')
                # 绘制最后一段线
                ax.plot([array_data[i-1, 0], array_data[i, 0]],
                        [array_data[i-1, 1], array_data[i, 1]],
                        [array_data[i-1, 2], array_data[i, 2]],
                        color=plt.cm.viridis(color_value), marker='o')
            else:
                ax.plot([array_data[i-1, 0], array_data[i, 0]], [array_data[i-1, 1], array_data[i, 1]], 
                        [array_data[i-1, 2], array_data[i, 2]],
                    color=plt.cm.viridis(color_value), marker='o')  # 使用RdPu颜色映射
        
        # 添加第一个黑色虚线圆圈
        theta = np.linspace(0, 2 * np.pi, 100)  # 圆的角度
        r = 0.025  # 圆的半径
        x_circle = 0.41025641 + r * np.cos(theta)
        y_circle = 0.41025641 + r * np.sin(theta)
        z_circle = np.full_like(theta, 0.17948718)  # 固定高度
        ax.plot(x_circle, y_circle, z_circle, 'k--')  # 黑色虚线圆圈

        # 自定义图例标记为黑色圆圈
        circle_legend = Line2D([0], [0], marker='o', color='k', label='Full score point',
                       markerfacecolor='w', markersize=10, linestyle='--')
        start_point_legend = Line2D([0], [0], marker='o', color='blue', label='Start Point', markersize=12)
        end_point_legend = Line2D([0], [0], marker='o', color='orange', label='End Point', markersize=12)

        # 添加图例
        ax.legend(handles=[start_point_legend,end_point_legend, circle_legend])

        # 添加第二个黑色虚线圆圈
        z_circle2 = np.full_like(theta, 0.0)  # 固定高度
        ax.plot(x_circle, y_circle, z_circle2, 'k--')  


        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_zlim(0, 1)        

        # 颜色归一化
        norm = Normalize(vmin=0, vmax=len(array_data) - 1)
        # 颜色映射
        cmap = plt.get_cmap('viridis')
        cmap = ScalarMappable(norm=norm, cmap=cmap)
        # 添加颜色条
        fig.colorbar(cmap, ax=ax, label='Round number')
        # 设置标签
        ax.set_xlabel('Relevant dimension 1', fontsize = 19)
        ax.set_ylabel('Relevant dimension 2', fontsize = 19)
        ax.set_zlabel('Non-relevant dimension', fontsize = 19)
        # ax.text(0.1,0.99, f'Highest score: {score_value}',fontsize = 18, color = 'crimson', ha='left', va='top',transform=ax.transAxes )
        # 设置标题
        ax.set_title(f'Exploration trace for {subject_value} highest score {score_value}.png')
        image_filename = os.path.join(save_dir, f'Exploration trace for {subject_value} highest score {score_value}.png')
        plt.savefig(image_filename)
        plt.show()
        plt.close(fig)




In [ ]:
#定义3D可视化的函数：(with score)
def plot_3d_trace_score(df, array_column_name, subject_column_name, score_column_name, save_dir):
    import os
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for index, row in df.iterrows():
        # 获取Subject值和对应的数组
        subject_value = row[subject_column_name]
        array_data = row[array_column_name]
        score_value = row[score_column_name]

        # 创建一个3D图形
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d') #降维到3维时改为3，降维到2维时改为2

        # 颜色归一化
        norm = Normalize(vmin=0, vmax=100)
        # 颜色映射
        cmap = plt.get_cmap('viridis')

        for i in range(len(array_data)):
            color_value = cmap(norm(score_value[i]))  # 根据分数设置颜色
            if i == 0:
                ax.scatter(array_data[i, 0], array_data[i, 1], array_data[i, 2], 
                        color=color_value, s=100, label='Start Point', marker='o')
            elif i == len(array_data) - 1:
                ax.scatter(array_data[i, 0], array_data[i, 1], array_data[i, 2], 
                        color=color_value, s=150, label='End Point', marker='o')
            else:
                ax.scatter(array_data[i, 0], array_data[i, 1], array_data[i, 2], 
                        color=color_value, s=50, marker='o')

        # 添加颜色条
        sm = ScalarMappable(norm=norm, cmap=cmap)
        sm.set_array(score_value)  # 设置颜色条的数据范围
        fig.colorbar(sm, ax=ax, label='Score')
        
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_zlim(0, 1)
        # 设置标签
        ax.set_xlabel('Food dim 1')
        ax.set_ylabel('Food dim2')
        ax.set_zlabel('Food dim3')
        # 设置标题
        ax.set_title(f'3D Trajectory of food dim variance for p1 {subject_value} ')
        # plt.legend()
        image_filename = os.path.join(save_dir, f'3D Trajectory of food dim variance for P1 {subject_value} with score.png')
        plt.savefig(image_filename)
        plt.close(fig)

In [ ]:
#定义一些函数，输入为combined_df中的probe/stick sequence

# task1 得到被试探索的维度的数量
def dim_probed(sequence):
    dim_sequence = [next(iter(item)) if isinstance(item, dict) else item for item in sequence] #转换为代表维度的数字
    int_dim_seq = [int(item) for item in dim_sequence] #转换为整型
    dim_probed = len(set([num for num in int_dim_seq if num != 0]))
    return dim_probed

# task2 得到sequence中每个“线段”的长度（在一个维度里连续探索的次数）列表
def probe_segments_length(sequence):
    dim_sequence = [next(iter(item)) if isinstance(item, dict) else item for item in sequence]  #转换为代表维度的数字
    int_dim_seq = [int(item) for item in dim_sequence]  #转换为整型
    result = []
    temp_list = []

    # 遍历原始列表
    for i, num in enumerate(int_dim_seq):
        if num != 0:  # 非零数字
            if not temp_list or temp_list[-1] == num:  # 当前段未开始或仍在同一段中
                temp_list.append(num)
            else:  # 开始新段
                result.append(len(temp_list))  # 记录上一段长度
                temp_list = [num]  # 重新初始化当前段
        else:  # 遇到0，保存当前段并清空
            if temp_list:
                result.append(len(temp_list))  # 记录小列表长度
                temp_list = []

    # 处理最后一个非零段
    if temp_list:
        result.append(len(temp_list))

    return result

# task3 得到sequence中每个维度探索的总排列数（跨“线段”的总数）---- 可以先不做


# task4 得到sequence中每个维度探索的次数（跨“线段”的总数）
def dim_probe_counts(sequence):
    from collections import Counter

    dim_sequence = [next(iter(item)) if isinstance(item, dict) else item for item in sequence]  #转换为代表维度的数字
    int_dim_seq = [int(item) for item in dim_sequence]  #转换为整型
    non_zero_counts = Counter(num for num in int_dim_seq if num != 0)
    result = list(non_zero_counts.values())
    return result



#test
sequence = combined_df1['Probe_sequence'][0]
print(sequence)
test_dimprobed = dim_probed(sequence)
print(test_dimprobed)
probe_segments = probe_segments_length(sequence)
print(probe_segments)
dim_probed_counts = dim_probe_counts(sequence)
print(dim_probed_counts)


In [ ]:
#返回probe sequence的函数
def probe_sequence(choice_sequence):
    probe_sequence_list = []
    data_int = choice_sequence.astype(int)
    for trial in data_int:
        round = np.array(trial, dtype=str)
        probe_rank = {}
        same_dim = {}
        rank = {}
        for blocks_idx, block in enumerate(round):
            num_small_objects = len(block[0])
            for i in range(num_small_objects):
                if block[0][i] == block[1][i] == block[2][i]:
                    same_dim[blocks_idx] = i
                    rank[blocks_idx] = block[0][i]
        if len(same_dim.keys()) == 3 and len(set(same_dim.values())) == 1 :
            probe_rank[same_dim[0]+1] = rank[0]+rank[1]+rank[2]
            probe_sequence_list.append(probe_rank)
        else: probe_sequence_list.append(0)
    return probe_sequence_list
        

In [ ]:
#计算probe fraction 函数
def probe_fraction(choice_sequence):
    data_int = choice_sequence.astype(int)
    test_list = []
    for trial in data_int:
        trial_if_test = is_testing_trial(trial)
        test_list.append(trial_if_test)
    count_ones = sum(test_list)
    total_length = len(test_list)
    fraction = count_ones / total_length
    return fraction

In [ ]:
#计算stick fraction 函数
def stick_fraction(choice_sequence):
    data_int = choice_sequence.astype(int)
    stick_list = []
    for trial in data_int:
        trial_if_stick = get_stick_trial(trial)
        stick_list.append(trial_if_stick)
    count_ones = sum(stick_list)
    total_length = len(stick_list)
    fraction = count_ones / total_length
    return fraction

In [ ]:
#返回stick sequence的函数
def stick_sequence(choice_sequence):
    stick_sequence_list = []
    data_int = choice_sequence.astype(int)
    for trial in data_int:
        round = np.array(trial, dtype=str)
        stick_food = {}
        same_dim = {}
        food_item = {}
        for blocks_idx, block in enumerate(round):
            num_small_objects = len(block[0])
            for i in range(num_small_objects):
                if block[0][i] == block[1][i] == block[2][i]:
                    same_dim[blocks_idx] = i
                    food_item[blocks_idx] = block[0][i]
        if len(same_dim.keys()) == 1 and list(same_dim.keys())[0] == 0 :
            stick_food[same_dim[0]+1] = food_item[0]
            stick_sequence_list.append(stick_food)
        else: stick_sequence_list.append(0)
    return stick_sequence_list

In [ ]:
#从文件名中提取被试名
def extract_key_From_filename(filename):
    key = filename.split('_test3')[0]
    prefix = key[:3].lower()
    if prefix =='oct':
        key = '10'+ key[3:]
    elif prefix == 'nov':
        key = '11' +key[3:]
    else:
        raise ValueError("Invalid month prefix") 

    return key

In [ ]:
def get_round_behavour(data):
    round_behavour =[]

    for i in range(len(data['block1_fullChoice'])):  #遍历csv_block数据框的每一行，用fullchoice这一列来确定数据框的行数
        B1=eval(data['block1_picChoice'].iloc[i].split('odict_keys')[1])
        #使用 split('odict_keys') 将字符串按 'odict_keys' 分割，并取分割后的第二部分（[1]），这部分数据应该是一个字符串格式的列表。
        #使用 eval() 将该字符串转换为实际的列表 B1。
        B1_int = list(map(int, B1))
        B2=eval(data['block2_picChoice'].iloc[i].split('odict_keys')[1])
        B2_int = list(map(int, B2))
        B3=eval(data['block3_picChoice'].iloc[i].split('odict_keys')[1])
        B3_int = list(map(int, B3))
        one_round=[B1_int,B2_int,B3_int]
        round_behavour.append(one_round)

    round_behavour=np.array(round_behavour)
    return round_behavour

In [ ]:
def is_testing_trial(round):
    same_dim = {}
    round = np.array(round, dtype=str)
    for blocks_idx, block in enumerate(round):
        num_small_objects = len(block[0])
        for i in range(num_small_objects):
            if block[0][i] == block[1][i] == block[2][i]:
                same_dim[blocks_idx] = i
    if len(same_dim.keys()) == 3 and len(set(same_dim.values())) == 1 :
        return  1
    else:
        return  0

In [ ]:
def get_stick_trial(round):
    same_dim = {}
    round = np.array(round, dtype=str)
    for blocks_idx, block in enumerate(round):
        num_small_objects = len(block[0])
        for i in range(num_small_objects):
            if block[0][i] == block[1][i] == block[2][i]:
                same_dim[blocks_idx] = i
    if len(same_dim.keys()) == 1 and list(same_dim.keys())[0] == 0 :
        return  1
    else:
        return  0

执行分析的功能cell：


In [ ]:
#读取原始csv数据，创建combined_df大表
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
directory_path1 = os.path.join(desktop_path, 'human_project_data', 'data_241118','human_data_forjiewen','p1_p2_csv','original_data')
directory_path2 = os.path.join(desktop_path, 'human_project_data', 'data_241118','human_data_forjiewen','p2_only_csv','original_data')
#创建初始dataframe（包含被试号，主特征，choice数组，分数数组，最高分数，反应时间数组（待加）

phase={}
mainfeature = {}
choice_sequence = {}
score_sequence = {}
Max_score = {}
for filename in sorted(os.listdir(directory_path1)):
    if filename.endswith('csv'):
        file_path = os.path.join(directory_path1, filename)
        csv = pd.read_csv(file_path)
        csv_block1 = csv[csv['phase']=='phase1']  #运行p2-only_group文件时改回csv_block
        score = csv_block1['reward'].values
        score_int = score.astype(int)
        choice_sequence1 = get_round_behavour(csv_block1)
        subject = extract_key_From_filename(filename)
        #使用正则表达式提取mainfeature原字符串中的数字作为新字符串
        mainfeature1 = csv_block1['p1Feature'].iloc[0] 
        mainfeature_num1 = re.findall(r'\d+', mainfeature1)
        mainfeature1 = ''.join(mainfeature_num1)
        max_score =str(np.max(score_int))
        phase[subject] = 'p1'
        mainfeature[subject] = mainfeature1
        choice_sequence[subject] = choice_sequence1
        score_sequence[subject] = score_int
        Max_score[subject] = max_score

# 将所有字典放在一个列表中
dict_list = [phase, mainfeature, choice_sequence, score_sequence, Max_score]
# 构建 DataFrame
df1 = pd.DataFrame(dict_list).T  # 转置使 key 成为第一列
df1.columns = ['Phase', 'Mainfeature', 'Choice_sequence', 'Score_sequence', 'Max_score']  # 设置列名
df1.reset_index(inplace=True)  # 重置索引，保留 key 作为第一列
df1.rename(columns={'index': 'Subject'}, inplace=True)  # 将 'index' 列重命名为 'Key'

phase={}
mainfeature = {}
choice_sequence = {}
score_sequence = {}
Max_score = {}
for filename in sorted(os.listdir(directory_path1)):
    if filename.endswith('csv'):
        file_path = os.path.join(directory_path1, filename)
        csv = pd.read_csv(file_path)
        csv_block1 = csv[csv['phase']=='phase1']
        csv_block2 = csv[csv['phase']=='phase2']  
        score = csv_block2['reward'].values
        score_int = score.astype(int)
        choice_sequence2 = get_round_behavour(csv_block2)
        subject = extract_key_From_filename(filename)
        #使用正则表达式提取mainfeature原字符串中的数字作为新字符串
        mainfeature2 = csv_block1['p2Feature'].iloc[0]
        mainfeature2 = str(mainfeature2)  
        mainfeature_num2 = re.findall(r'\d+', mainfeature2)
        mainfeature2 = ''.join(mainfeature_num2)
        max_score =str(np.max(score_int))
        phase[subject] = 'p2'
        mainfeature[subject] = mainfeature2
        choice_sequence[subject] = choice_sequence2
        score_sequence[subject] = score_int
        Max_score[subject] = max_score
# 将所有字典放在一个列表中
dict_list = [phase, mainfeature, choice_sequence, score_sequence, Max_score]
# 构建 DataFrame
df2 = pd.DataFrame(dict_list).T  # 转置使 key 成为第一列
df2.columns = ['Phase', 'Mainfeature', 'Choice_sequence', 'Score_sequence', 'Max_score']  # 设置列名
df2.reset_index(inplace=True)  # 重置索引，保留 key 作为第一列
df2.rename(columns={'index': 'Subject'}, inplace=True)  # 将 'index' 列重命名为 'Key'

phase={}
Mainfeature = {}
Choice_sequence = {}
score_sequence = {}
Max_score = {}
for filename in sorted(os.listdir(directory_path2)):
    if filename.endswith('csv'):
        file_path = os.path.join(directory_path2, filename)
        csv = pd.read_csv(file_path)
        csv_block = csv[csv['phase']=='phase2']  
        score = csv_block['reward'].values
        score_int = score.astype(int)
        choice_sequence = get_round_behavour(csv_block)
        subject = extract_key_From_filename(filename)
        #使用正则表达式提取mainfeature原字符串中的数字作为新字符串
        mainfeature = csv_block['mainFeature'].iloc[0] 
        mainfeature_num = re.findall(r'\d+', mainfeature)
        mainfeature = ''.join(mainfeature_num)
        max_score =str(np.max(score_int))
        phase[subject] = 'p2only'
        Mainfeature[subject] = mainfeature
        Choice_sequence[subject] = choice_sequence
        score_sequence[subject] = score_int
        Max_score[subject] = max_score

# 将所有字典放在一个列表中
dict_list = [phase, Mainfeature, Choice_sequence, score_sequence, Max_score]
# 构建 DataFrame
df3 = pd.DataFrame(dict_list).T  # 转置使 key 成为第一列
df3.columns = ['Phase', 'Mainfeature', 'Choice_sequence', 'Score_sequence', 'Max_score']  # 设置列名
df3.reset_index(inplace=True)  # 重置索引，保留 key 作为第一列
df3.rename(columns={'index': 'Subject'}, inplace=True)  # 将 'index' 列重命名为 'Key'

print(f"The shape of the df1 is: {df1.shape}")
print(f"The shape of the df2 is: {df2.shape}")
print(f"The shape of the df3 is: {df3.shape}")

combined_df = pd.concat([df1, df2, df3], ignore_index=True)
data = combined_df['Choice_sequence']
choice_seq = []
for item in data:
    choice_seq.append(item)

In [ ]:
#将choice sequence按food item 抽提整理为表格形式
import pandas as pd
import numpy as np
from openpyxl import Workbook

def sort_digits(number, mainfeature):
    digits = str(number)
    mainfeature = str(mainfeature)
    # 根据mainfeature的数字对数位进行排序
    flag = np.zeros(len(digits))
    index = []
    for digit in mainfeature:
        index.append(int(digit)-1)
        flag[int(digit)-1] = 1
    for i, item in enumerate(flag):
        if item == 0:
            index.append(i)
    # 将排序后的数位重新组合成数字
    digits = [digits[i] for i in index]
    sorted_number = int(''.join(map(str, digits)))
    return sorted_number

def process_choice_sequence(choice_sequence, mainfeature):
    _draw_digit = lambda x, i: str(x)[i] 
    flag = np.zeros(len(str(choice_sequence[0,0,0])))
    index = []
    for digit in mainfeature:
        index.append(int(digit)-1)
        flag[int(digit)-1] = 1
    for i, item in enumerate(flag):
        if item == 0:
            index.append(i)

    collect_round = []
    for round in choice_sequence:
        collect_feature = []
        for i in index:
            collect_phase = []
            for phase in round:
                collect_phase.append(f'{_draw_digit(phase[0], i)}{_draw_digit(phase[1], i)}{_draw_digit(phase[2], i)}')
            collect_feature.append(collect_phase)
        collect_round.append(collect_feature)
    return collect_round

# 处理DataFrame中的每一行
# df['Processed_Choice_sequence'] = df.apply(lambda row: process_choice_sequence(row['Choice_sequence'], row['Mainfeature']), axis=1)

wb = Workbook()
tmp = wb.active

for index, row in combined_df.iterrows():
    mainfeature = row['Mainfeature']
    score = row['Score_sequence']
    choice_subject = choice_seq[index]
    output = process_choice_sequence(choice_subject, mainfeature)
    output = np.array(output)

    table = []
    for i in range(output.shape[0]):
        table_line = []
        for j in range(output.shape[1]):
            table_line.append(f'{output[i, j, 0]} {output[i, j, 1]} {output[i, j, 2]} ')
            
        table_line.append(f'{int(score[i])}')
        table.append(table_line)
        

    phase_subject = f"{row['Subject']}_{row['Phase']}"  # 文件名
    ws = wb.create_sheet(phase_subject)

    for item in table:
        ws.append(item)

wb.save("rearrange_data.xlsx")

In [ ]:
#计算 probe sequence , probe fraction , stick sequence, stick fraciton加入大表作为新列
combined_df['Probe_sequence'] = combined_df['Choice_sequence'].apply(probe_sequence)
combined_df['Probe_fraction'] = combined_df['Choice_sequence'].apply(probe_fraction)
combined_df['Stick_sequence'] = combined_df['Choice_sequence'].apply(stick_sequence)
combined_df['Stick_fraction'] = combined_df['Choice_sequence'].apply(stick_fraction)


In [ ]:
#计算Food_index_arr,Variance_arr, ---- for p2 p2only
data_choicep1 = combined_df['Choice_sequence'][combined_df['Phase'] == 'p1']
data_choicep2 = combined_df['Choice_sequence'][combined_df['Phase'] == 'p2']
data_choicep2only = combined_df['Choice_sequence'][combined_df['Phase'] == 'p2only']

data_p2_p2only = pd.concat([data_choicep2, data_choicep2only])
#data_p2p2only_int = data_p2_p2only.astype(int)
choice_data = data_p2_p2only.to_numpy()
#包含p2p2only所有被试的food preference index ndarray的大列表
food_index_list = [food_preference_arr(choice) for choice in choice_data] 
#将列表中所有的ndarray在垂直方向合并为大array用于统一做pca
combined_array = np.vstack(food_index_list)

pca = PCA(n_components = 3)  #降维到3维时改为3，降维到2维时改为2
array_data = pca.fit_transform(combined_array)
pca_weights = pca.components_  # Shape: (3, 12)

# 打印权重
for i, component in enumerate(pca_weights):
    print(f"PC{i+1} weights: {component}")
    
#使用包含p2p2only所有被试的food preference index ndarray的大列表，创建有被试号Subject列对应的两列的dataframe
#对food preference index列计算得到variance数组（x,x,x,x)对应1，2，3，4维度

#获取被试号列表
Subject_series_p2 = combined_df['Subject'][combined_df['Phase'] == 'p2']
Subject_series_p2only = combined_df['Subject'][combined_df['Phase'] == 'p2only']
Subject_series_p2p2only = pd.concat([Subject_series_p2, Subject_series_p2only], ignore_index=True)
print(len(Subject_series_p2p2only), len(food_index_list))
#构建包含‘Subject'列和‘Food_index_arr'两列的dataframe
p2food_index_df = pd.DataFrame({
    'Subject': Subject_series_p2p2only,  # 第一列
    'Food_index_arr': food_index_list     # 第二列
})
#对Food_index_arr列应用函数得到Variance_arr列
p2food_index_df['Variance_arr'] = p2food_index_df['Food_index_arr'].apply(get_dim_variance)


In [ ]:
#计算Food_index_arr,Variance_arr, ---- for p1
data_choicep1 = combined_df['Choice_sequence'][combined_df['Phase'] == 'p1']
choice_data = data_choicep1.to_numpy()
#包含p1所有被试的food preference index ndarray的大列表
food_index_list_p1 = [food_preference_arr(choice) for choice in choice_data] 
#将列表中所有的ndarray在垂直方向合并为大array用于统一做pca
combined_array = np.vstack(food_index_list_p1)

pca = PCA(n_components = 3)  #降维到3维时改为3，降维到2维时改为2
array_data = pca.fit_transform(combined_array)
pca_weights = pca.components_  # Shape: (3, 9)

# 打印权重
for i, component in enumerate(pca_weights):
    print(f"PC{i+1} weights: {component}")
    
#使用包含p1所有被试的food preference index ndarray的大列表，创建有被试号Subject列对应的两列的dataframe
#对food preference index列计算得到variance数组（x,x,x)对应1，2，3维度

#获取被试号列表
Subject_series_p1 = combined_df['Subject'][combined_df['Phase'] == 'p1']
Mainfeature_series_p1 = combined_df['Mainfeature'][combined_df['Phase'] == 'p1']
Score_series_p1 = combined_df['Score_sequence'][combined_df['Phase'] == 'p1']
#将food_index_list_p1中食物顺序按照主特征字符串重新排序，主特征食物在前面6位
def reorder_array_by_string(arr_list, order_str):
    reordered_list = []
    for idx, arr in enumerate(arr_list):
        reordered_arr_subject = []
        for round in arr:
            # 将数组分组为3组
            groups = [round[:3], round[3:6], round[6:]]
            
            # 根据字符串中的数字顺序重新排列分组
            included_groups = [groups[int(i) - 1] for i in order_str[idx]]
            excluded_groups = [groups[i] for i in range(3) if str(i + 1) not in order_str[idx]]
            
            # 合并包含的组和未包含的组
            reordered_groups = included_groups + excluded_groups
            
            # 将重新排列的分组合并成一个数组
            reordered_arr = [num for group in reordered_groups for num in group]
            
            reordered_arr_subject.append(reordered_arr)
        reordered_list.append(reordered_arr_subject)

    return reordered_list

reordered_food_index_list_p1 = reorder_array_by_string(food_index_list_p1,Mainfeature_series_p1 )

print(len(Subject_series_p1), len(food_index_list_p1))
#构建包含‘Subject'列和‘Food_index_arr'两列的dataframe
p1food_index_df = pd.DataFrame({
    'Subject': Subject_series_p1,  # 第一列
    'Food_index_arr_reordered': reordered_food_index_list_p1,  # 第二列
    'Score_sequence':Score_series_p1
})

#对Food_index_arr列应用函数得到Variance_arr列
p1food_index_df['Variance_arr'] = p1food_index_df['Food_index_arr_reordered'].apply(get_dim_variance)

#定义文件保存路径
fig_save_dir1 = os.path.join(desktop_path, 'food dim variance plot test','for p1')
fig_save_dir2 = os.path.join(desktop_path, 'food dim variance plot test','for p1 with score')
#3D 可视化
plot_3d_trace(p1food_index_df, 'Variance_arr', 'Subject','Score_sequence', fig_save_dir1 )
#plot_3d_trace_score(p1food_index_df, 'Variance_arr', 'Subject','Score_sequence', fig_save_dir2 )


In [ ]:
#从endsurvey indicator 文件中导入food dim if correct1 , spearman corr, identified dim number, 加到combined_df中p2和p2only的数据,作为新df：df_analysis1
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
directory_path = os.path.join(desktop_path, 'human_project_data','indicator_csv')
file_path1 = os.path.join(directory_path, 'p2 food dim answer indicator.csv')
file_path2 = os.path.join(directory_path, 'p2only food dim answer indicator.csv')
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
p2_data = combined_df[combined_df['Phase'] == 'p2']
p2only_data = combined_df[combined_df['Phase'] == 'p2only']

p2_data['Subject'] = p2_data['Subject'].astype(str)  # 或者 int，根据数据情况
p2only_data['Subject'] = p2only_data['Subject'].astype(str)
df1['Subject'] = df1['Subject'].astype(str)  # 确保与 combined_df 一致
df2['Subject'] = df2['Subject'].astype(str)  # 确保与 combined_df 一致

df_analysis_p2 = pd.merge(p2_data, df1[['Subject', 'If correct1','Correlation coefficient','Identified number']], on='Subject', how='left')
df_analysis_p2only = pd.merge(p2only_data, df2[['Subject', 'If correct1','Correlation coefficient','Identified number']], on='Subject', how='left')
df_analysis1 = pd.concat([df_analysis_p2, df_analysis_p2only])

In [ ]:
# fraction of people had correct answer for each category of food type mentioning
#for p2 p2only data, identified number missing value 的就先去掉了
mention_one = df_analysis1[df_analysis1['Identified number']== 1]
mention_two = df_analysis1[df_analysis1['Identified number']== 2]
mention_three = df_analysis1[df_analysis1['Identified number']== 3]
mention_four = df_analysis1[df_analysis1['Identified number']== 4]
mention_one_correct = mention_one[mention_one['If correct1']==1]
mention_two_correct = mention_two[mention_two['If correct1']==1]
mention_three_correct = mention_three[mention_three['If correct1']==1]
mention_four_correct = mention_four[mention_four['If correct1']==1]
fraction1 = len(mention_one_correct) / len(mention_one)
fraction2 = len(mention_two_correct) / len(mention_two)
fraction3 = len(mention_three_correct) / len(mention_three)
fraction4 = len(mention_four_correct) / len(mention_four)
print(len(mention_one_correct))
print(len(mention_one))
print(len(mention_two_correct))
print(len(mention_two))
print(len(mention_three_correct))
print(len(mention_three))
print(len(mention_four_correct) )
print(len(mention_four))

#FPR-food dim, 2group
x1 = fraction1
x2 = fraction2
x3 = fraction3
x4 = fraction4
values = [x1, x2, x3, x4]
percentages = [x * 100 for x in values]
labels = ['correct with one dim mentioned','correct with two dim mentioned','correct with three dim mentioned','correct with four dim mentioned' ]
fig,ax = plt.subplots()
bars = plt.bar(labels, percentages, color = [ 'darkgrey','gainsboro','dimgrey','silver'],hatch=['**','**','**','**'])
plt.title('Comparison of fraction correct for 4 category of dimention identification')
plt.xlabel('Conditions')
plt.ylabel('Percentage of fraction correct')
#plt.ylim(0,50)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 1), ha='center', va='bottom')
plt.show()


In [ ]:
# conditional analysis on full score or correct identification of types—>complete both true and null conditions
df_analysis1['If correct1'] = df_analysis1['If correct1'].fillna(0)
full_score_correct = df_analysis1[(df_analysis1['Max_score'] == '100') & (df_analysis1['If correct1']== 1)]
full_score_not_correct = df_analysis1[(df_analysis1['Max_score'] == '100') & (df_analysis1['If correct1']!= 1)]
not_full_score_correct = df_analysis1[(df_analysis1['Max_score'] != '100') & (df_analysis1['If correct1']== 1)]
not_full_score_not_correct = df_analysis1[(df_analysis1['Max_score'] != '100') & (df_analysis1['If correct1']!= 1)]
Tp = len(full_score_correct.to_numpy())
Fn = len(not_full_score_not_correct.to_numpy())
Fp = len(not_full_score_correct.to_numpy())
Tn = len(full_score_not_correct.to_numpy())
print(f'tp={Tp},fn={Fn},fp={Fp},tn={Tn}')
conf_matrix =  np.array([[Tp,Tn],
                         [Fp,Fn]])

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['answer correct', 'answer not correct'], 
            yticklabels=['full score', 'not fullscore'])

# 添加标题
plt.title('Confusion Matrix of if full score and answer if correct for all subject in p2 and p2only',fontsize=14)
plt.xlabel("answer if correct")
plt.ylabel('if full score')
plt.show()



In [ ]:
# 对correct / not correct 画stick fraction小提琴图 ---------没有显著性差异
datap2 = df_analysis1[df_analysis1['Phase'] == 'p2']
stickfraction_correct  = pd.to_numeric(datap2['Probe_fraction'][datap2['If correct1']==1])
stickfraction_not_correct = pd.to_numeric(datap2['Probe_fraction'][datap2['If correct1']==0])

group1 = stickfraction_correct.to_numpy()
group2 = stickfraction_not_correct.to_numpy() 

df = pd.DataFrame({
    "Group": ["Answer correct"] * len(group1) + ["Answer not correct"] * len(group2),
    "Value": np.concatenate([group1, group2]),
    "ColorGroup": ["Answer correct"] * len(group1) + ["Answer not correct"] * len(group2)  # 添加颜色分组
})
plt.figure(figsize=(6, 5))
sns.violinplot(x="Group", y="Value", data=df, inner=None, color='pink', linewidth=1.5, alpha=0.2)
sns.stripplot(x="Group", y="Value", data=df, jitter=True, size=5, hue="ColorGroup", palette={"Answer correct": "grey", "Answer not correct": "gainsboro"}, alpha=0.8)

for group in df["Group"].unique():
    group_values = df[df["Group"] == group]["Value"]
    median = np.median(group_values)
    mean = np.mean(group_values)
    q1 = np.percentile(group_values, 25)
    q3 = np.percentile(group_values, 75)
    
    x_pos = list(df["Group"].unique()).index(group)

    plt.scatter(x_pos, median, color="red", label="Median" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, mean, color="green", label="Mean" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q1, color="purple", label="Q1 (25th Percentile)" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q3, color="orange", label="Q3 (75th Percentile)" if group == df["Group"].unique()[0] else "")

plt.legend()
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.title("Stick fraction for subject with correct/not correct answer", fontsize=16)
plt.xlabel("Group", fontsize=14)
plt.ylabel("Stick fraction", fontsize=14)
plt.show()

t_stat, p_value = ttest_ind(group1, group2, equal_var=False)  # 设置 equal_var=False 假设方差不相等
print(f"T-test: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")

# 判断显著性
if p_value < 0.05:
    print("两组之间的差异具有统计学意义（p < 0.05）。")
else:
    print("两组之间的差异不具有统计学意义（p >= 0.05）。")

In [ ]:
#从endsurvey indicator 文件中导入food item spearman corr, identified food number, 加到combined_df中,作为新df：df_analysis2

In [ ]:
#从原始probe sequence中得到用分数排序并提取了probe维度数字的sequence
def convert_probe_list_for_food_item(probe_list):
    return [next(iter(item.values()))[0] if isinstance(item, dict) else item for item in probe_list]
combined_df1 = combined_df[combined_df['Phase'] == 'p1'].sort_values(by='Max_score', ascending=True)
all_probe_list_p1 = combined_df1['Stick_sequence'] 
probe_food_list_p1 = [convert_probe_list_for_food_item(probe_list) for probe_list in all_probe_list_p1]
combined_df2 = combined_df[combined_df['Phase'] == 'p2'].sort_values(by='Max_score', ascending=True)
all_probe_list_p2 = combined_df2['Stick_sequence'] 
probe_food_list_p2 = [convert_probe_list_for_food_item(probe_list) for probe_list in all_probe_list_p2]
combined_df2only = combined_df[combined_df['Phase'] == 'p2only'].sort_values(by='Max_score', ascending=True)
all_probe_list_p2only = combined_df2only['Stick_sequence'] 
probe_food_list_p2only = [convert_probe_list_for_food_item(probe_list) for probe_list in all_probe_list_p2only]

In [ ]:
#用probe sequence 画热图可视化probe的维度, 画stick sequence时进行替换

# combined_df1 = combined_df[combined_df['Phase'] == 'p1'].sort_values(by='Max_score', ascending=True)
# combined_df2 = combined_df[combined_df['Phase'] == 'p2'].sort_values(by='Max_score', ascending=True)
# combined_df2only = combined_df[combined_df['Phase'] == 'p2only'].sort_values(by='Max_score', ascending=True)


# all_probe_list_p1 = combined_df1['Stick_sequence'] 
subject_list_p1 = combined_df1['Subject']
mainfeature_list_p1 = combined_df1['Mainfeature']

# all_probe_list_p2 = combined_df2['Stick_sequence']
subject_list_p2 = combined_df2['Subject']
mainfeature_list_p2 = combined_df2['Mainfeature']

# all_probe_list_p2only = combined_df2only['Stick_sequence']
subject_list_p2only = combined_df2only['Subject']
mainfeature_list_p2only = combined_df2only['Mainfeature']


#将list中的字典元素转换为字典key的值，0保持为0
def convert_probe_list(probe_list):
    return [next(iter(item)) if isinstance(item, dict) else item for item in probe_list]



# 修改all_probe_list_p1, all_probe_list_p2, all_probe_list_p2only
all_probe_list_p1 = [convert_probe_list(probe_list) for probe_list in all_probe_list_p1]
all_probe_list_p2 = [convert_probe_list(probe_list) for probe_list in all_probe_list_p2]
all_probe_list_p2only = [convert_probe_list(probe_list) for probe_list in all_probe_list_p2only]

# 为每个列表补齐长度并转换为ndarray
max_length1 = max(len(lst) for lst in all_probe_list_p1)
end_p1 = [len(lst) for lst in all_probe_list_p1]
data_visualize1 = [lst + [0] * (max_length1 - len(lst)) for lst in all_probe_list_p1]
food_visualize1 = [lst + [0] * (max_length1 - len(lst)) for lst in probe_food_list_p1]
data_array_p1 = np.array(data_visualize1) # data_array_ 是最终用来画图的数据
food_array_p1 = np.array(food_visualize1)
food_array_p1_skip0 = np.where(food_array_p1 == '0', ' ', food_array_p1)
#制作ystick array
subject_array_p1 = np.array(subject_list_p1)
mainfeature_array_p1 = np.array(mainfeature_list_p1) # 可用于判断probe的维度是否为重要维度
ystick_arr_p1 = subject_array_p1 + '_' + mainfeature_array_p1
#制作终点标记 array
end_arr_p1 = np.array(end_p1)

max_length2 = max(len(lst) for lst in all_probe_list_p2)
end_p2 = [len(lst) for lst in all_probe_list_p2]
data_visualize2 = [lst + [0] * (max_length2 - len(lst)) for lst in all_probe_list_p2]
data_array_p2 = np.array(data_visualize2) # data_array_ 是最终用来画图的数据
food_visualize2 = [lst + [0] * (max_length2 - len(lst)) for lst in probe_food_list_p2]
food_array_p2 = np.array(food_visualize2)
food_array_p2_skip0 = np.where(food_array_p2 == '0', ' ', food_array_p2)
subject_array_p2 = np.array(subject_list_p2)
mainfeature_array_p2 = np.array(mainfeature_list_p2) # 可用于判断probe的维度是否为重要维度
ystick_arr_p2 = subject_array_p2 + '_' + mainfeature_array_p2
end_arr_p2 = np.array(end_p2)

max_length3 = max(len(lst) for lst in all_probe_list_p2only)
end_p2only = [len(lst) for lst in all_probe_list_p2only]
data_visualize3 = [lst + [0] * (max_length3 - len(lst)) for lst in all_probe_list_p2only]
data_array_p2only = np.array(data_visualize3)  # data_array_ 是最终用来画图的数据
food_visualize2only = [lst + [0] * (max_length3 - len(lst)) for lst in probe_food_list_p2only]
food_array_p2only = np.array(food_visualize2only)
food_array_p2only_skip0 = np.where(food_array_p2only == '0', ' ', food_array_p2only)
subject_array_p2only = np.array(subject_list_p2only)
mainfeature_array_p2only = np.array(mainfeature_list_p2only) # 可用于判断probe的维度是否为重要维度
ystick_arr_p2only = subject_array_p2only + '_' + mainfeature_array_p2only
end_arr_p2only = np.array(end_p2only)

#heatmap for p1/p2/p2only (画的时候换参数)
fig, ax = plt.subplots(figsize=(60, 15))  # 设置图像大小
cax = ax.matshow(data_array_p2, cmap="viridis")  # 使用 'viridis' 颜色映射，也可以换成其他 cmap
cbar = plt.colorbar(cax, ax=ax)
cbar.set_label('Values (0-4)', rotation=270, labelpad=15)  # 设置颜色条标题

# 在热图上标注数字
for i in range(data_array_p2.shape[0]):
    for j in range(data_array_p2.shape[1]):
        ax.text(j, i, str(food_array_p2_skip0[i, j]), ha='center', va='center', color='white', fontsize=16)
#终点标记
ax.scatter(end_arr_p2, range(len(end_arr_p2)), color="crimson", label="End Points", s=100, edgecolors="black")
# 设置坐标轴
ax.set_xticks(range(data_array_p2.shape[1]))
ax.set_yticks(range(data_array_p2.shape[0])) 
ax.set_yticklabels(ystick_arr_p2, fontsize=15)
ax.set_xlabel('Trial number', fontsize=15)
ax.set_ylabel('Subject_Mainfeature', fontsize=15)
ax.set_title('visualization of three-block probe dim for p2', fontsize=16)

plt.show()

In [ ]:
#在上一个cell 的基础上，根据Mainfeature,把主特征改为2，非主特征改为4 
#数据：data_array_p1，data_array_p2，data_array_p2only

def main_nonmain_feature(arr_2d, arr_1d):
    arr = np.zeros(arr_2d.shape, dtype=int)
    for i in range(arr_2d.shape[0]):  # 遍历二维数组的行
        unmatched_count = []
        for j in range(arr_2d.shape[1]):  # 遍历每行中的元素
            current_value = str(arr_2d[i, j])
            compare_str = arr_1d[i]
            if current_value == '0':
                arr[i,j] = 0
            elif current_value in compare_str:
                # 如果当前值在对应行的字符串中
                if current_value == compare_str[0]:
                    arr[i, j] = 1  # 如果是第一位数字
                elif current_value == compare_str[1]:
                    arr[i, j] = 2  # 如果是第二位数字
            else:
                unmatched_count.append(current_value)
                array = np.array(unmatched_count)
                array_unique = np.unique(array)
                if len(array_unique) == 1:
                    arr[i,j] = 3
                elif len(array_unique) == 2:
                    if current_value == array[0]:
                        arr[i,j] = 3
                    else: arr[i,j] = 4
    return arr


#对p1 data进行revert
data_p1_revert = main_nonmain_feature(data_array_p1,mainfeature_array_p1) # 是用于画图的数据
# #p2
data_p2_revert = main_nonmain_feature(data_array_p2,mainfeature_array_p2) # 是用于画图的数据
#p2only
data_p2only_revert = main_nonmain_feature(data_array_p2only,mainfeature_array_p2only)

#画图部分
colors = ['snow', '#FFCCCB', '#FF8C00', '#87CEFA','#0000FF']  # 灰色、暖色（浅红和橙色）、冷色（浅蓝和深蓝） #p2p2only加颜色 '#0000FF'
cmap = ListedColormap(colors)
bounds = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5 ]  # 每个颜色的边界  #p2p2only加边界 4.5
norm = BoundaryNorm(bounds, cmap.N)

plt.figure(figsize=(9, 7))
heatmap = plt.imshow(data_p2only_revert, cmap=cmap, norm=norm, aspect='auto') #p1p2p2only

# 添加颜色图例
colorbar = plt.colorbar(heatmap, ticks=[0, 1, 2, 3, 4])        #p1 p2p2only 改
colorbar.set_label('Value', fontsize=12)
colorbar.ax.set_yticklabels(['non-probe', 'relevant dim1', 'relevant dim2', 
                             'irrelevant dim1','irrelevant dim2'])            #p1 p2p2only 改



# 设置标题和轴标签
plt.title('Heatmap of one-block probe bahavior for p2only group', fontsize=16)  #p1p2p2only
plt.xlabel('Rounds', fontsize=14)
plt.ylabel('Subjects', fontsize=14)

# 设置刻度
ax = plt.gca()  # 获取当前 Axes
#标注probe的该维度里的具体食物（第一排）
for i in range(data_array_p2only.shape[0]): #p1p2p2only
    for j in range(data_array_p2only.shape[1]):  #p1p2p2only
        ax.text(j, i, str(food_array_p2only_skip0[i, j]), ha='center', va='center', color='firebrick', fontsize=10)  #p1p2p2only
ax.set_xticks(range(data_p2only_revert.shape[1])) #p1p2p2only
ax.set_yticks(range(data_p2only_revert.shape[0])) #p1p2p2only
ax.set_yticklabels([f'{subject_array_p2only[i]}' for i in range(data_p2only_revert.shape[0])], fontsize=10)  # 自定义 y 轴标签  #两处p1p2p2only
ax.set_xticklabels([f'{j+1}' for j in range(data_p2only_revert.shape[1])], fontsize=10, rotation=90)  # 自定义 x 轴标签  #p1p2p2only
ax.scatter(end_arr_p2only, range(len(end_arr_p2only)), color="green", label="End Points", s=50, edgecolors="black")  #两处p1p2p2only
# 显示图表
plt.tight_layout()
plt.show()



In [ ]:
# 将 P1 endsurvey 中的indicator加入combined_p1 df中作为 analysis_p1
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
directory_path = os.path.join(desktop_path, 'human_project_data','indicator_csv')
file_path = os.path.join(directory_path, 'p1 food item answer indicator.csv')
df = pd.read_csv(file_path)
p1_data = combined_df[combined_df['Phase'] == 'p1']

p1_data['Subject'] = p1_data['Subject'].astype(str)  # 或者 int，根据数据情况
df['Subject'] = df['Subject'].astype(str)  # 确保与 combined_df 一致

df_analysis_p1 = pd.merge(p1_data, df[['Subject', 'Correlation coefficient','False positive rate','Identified number']], on='Subject', how='left')
df_analysis_p1 = df_analysis_p1.dropna(subset=['Correlation coefficient'])

from scipy.stats import pearsonr
sns.reset_defaults()

x = pd.to_numeric(df_analysis_p1['Stick_fraction'], errors='coerce')
y = pd.to_numeric(df_analysis_p1['Correlation coefficient'], errors='coerce')
corr, p_value = pearsonr(x, y)#计算斯皮尔曼相关系数
data = pd.DataFrame({"x": x, "y": y})
data["count"] = data.groupby(["x", "y"])["x"].transform("count")

fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot(x="x", y="y", size="count", sizes=(20, 200), hue="count", palette="viridis", data=data, ax=ax, legend="brief")
ax.legend(
    title="Count",          # 图例标题
    bbox_to_anchor=(1.05, 1),  # 图例框位置 (x, y)
    loc="upper left",       # 图例锚点
    borderaxespad=0.        # 图例与轴之间的间距
)

sns.regplot(x=x, y=y, ax=ax, scatter=False, line_kws={'color': 'grey'})
ax.set_title(f'Answer corr with stick fraction for p1 group', fontsize = 15)
ax.set_xlabel( " stick fraction ",fontsize = 14)
ax.set_ylabel('Answer corr',fontsize = 14)
ax.text(0.1,0.99, f'Pearson corr: {corr:.2f},P-value:{p_value:.2f}',fontsize = 13, color = 'grey', ha='left', va='top',transform=ax.transAxes )
ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.show()


In [ ]:
#p1 answer false positive rate/corr with probe/stick fration
from scipy.stats import pearsonr
sns.reset_defaults()

df_analysis_p1 = df_analysis_p1.dropna(subset=['False positive rate'])

x = pd.to_numeric(df_analysis_p1['Stick_fraction'], errors='coerce')
y = pd.to_numeric(df_analysis_p1['False positive rate'], errors='coerce')
corr, p_value = spearmanr(x, y)#计算斯皮尔曼相关系数
data = pd.DataFrame({"x": x, "y": y})
data["count"] = data.groupby(["x", "y"])["x"].transform("count")

fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot(x="x", y="y", size="count", sizes=(20, 200), hue="count", palette="viridis", data=data, ax=ax, legend="brief")
ax.legend(
    title="Count",          # 图例标题
    bbox_to_anchor=(1.05, 1),  # 图例框位置 (x, y)
    loc="upper left",       # 图例锚点
    borderaxespad=0.        # 图例与轴之间的间距
)

sns.regplot(x=x, y=y, ax=ax, scatter=False, line_kws={'color': 'grey'})
ax.set_title(f'FPR with stick fraction for p1 group', fontsize = 15)
ax.set_xlabel( " Stick fraction ",fontsize = 14)
ax.set_ylabel('FPR',fontsize = 14)
ax.text(0.1,0.99, f'Spearman corr: {corr:.2f},P-value:{p_value:.2f}',fontsize = 13, color = 'grey', ha='left', va='top',transform=ax.transAxes )
ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.show()

In [ ]:
#probe fraction with score
from scipy.stats import pearsonr
from scipy.stats import spearmanr
sns.reset_defaults()

x = pd.to_numeric(combined_df1['Stick_fraction'], errors='coerce')
y = pd.to_numeric(combined_df1['Max_score'], errors='coerce')
corr, p_value = spearmanr(x, y)#计算斯皮尔曼相关系数
data = pd.DataFrame({"x": x, "y": y})
data["count"] = data.groupby(["x", "y"])["x"].transform("count")

fig, ax = plt.subplots(figsize=(5,5))
sns.scatterplot(x="x", y="y", size="count", sizes=(20, 200), hue="count", palette="viridis", data=data, ax=ax, legend="brief")
ax.legend(
    title="Count",          # 图例标题
    bbox_to_anchor=(1.05, 1),  # 图例框位置 (x, y)
    loc="upper left",       # 图例锚点
    borderaxespad=0.        # 图例与轴之间的间距
)

sns.regplot(x=x, y=y, ax=ax, scatter=False, line_kws={'color': 'grey'})
ax.set_title(f'Stick fraction with score for p1 group all subject', fontsize = 15)
ax.set_xlabel( " One-block probe fraction ",fontsize = 14)
ax.set_ylabel('The highest score of subject',fontsize = 14)
ax.text(0.1,0.99, f'Pearson corr: {corr:.2f},P-value:{p_value:.2f}',fontsize = 13, color = 'grey', ha='left', va='top',transform=ax.transAxes )
ax.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.show()

# 提取non-stick subject的最高分数，stick subject的最高分数，用提亲图画分数分布和均值，作统计检验
score_non_stick  = pd.to_numeric(combined_df1['Max_score'][combined_df['Stick_fraction']==0])
score_stick = pd.to_numeric(combined_df['Max_score'][combined_df['Stick_fraction'] != 0])

group1 = score_non_stick.to_numpy()
group2 = score_stick.to_numpy() 

df = pd.DataFrame({
    "Group": ["non_stick"] * len(group1) + ["stick"] * len(group2),
    "Value": np.concatenate([group1, group2]),
    "ColorGroup": ["non_stick"] * len(group1) + ["stick"] * len(group2)  # 添加颜色分组
})
plt.figure(figsize=(6, 5))
sns.violinplot(x="Group", y="Value", data=df, inner=None, color='pink', linewidth=1.5, alpha=0.1)
sns.stripplot(x="Group", y="Value", data=df, jitter=True, size=5, hue="ColorGroup", palette={"non_stick": "grey", "stick": "gainsboro"}, alpha=0.8)

for group in df["Group"].unique():
    group_values = df[df["Group"] == group]["Value"]
    median = np.median(group_values)
    mean = np.mean(group_values)
    q1 = np.percentile(group_values, 25)
    q3 = np.percentile(group_values, 75)
    
    x_pos = list(df["Group"].unique()).index(group)

    plt.scatter(x_pos, median, color="red", label="Median" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, mean, color="green", label="Mean" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q1, color="purple", label="Q1 (25th Percentile)" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q3, color="orange", label="Q3 (75th Percentile)" if group == df["Group"].unique()[0] else "")

plt.legend()
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.title("Maxscore for subjects with/without sticking behavior", fontsize=16)
plt.xlabel("Group", fontsize=14)
plt.ylabel("Counts", fontsize=14)
plt.show()

t_stat, p_value = ttest_ind(group1, group2, equal_var=False)  # 设置 equal_var=False 假设方差不相等
print(f"T-test: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")

# 判断显著性
if p_value < 0.05:
    print("两组之间的差异具有统计学意义（p < 0.05）。")
else:
    print("两组之间的差异不具有统计学意义（p >= 0.05）。")


In [ ]:
#probe fraction/stick fraction for subject with full/nonfull score for p2
#p2_data = combined_df[combined_df['Phase']== 'p2only']
probe_fraction_full_score  = pd.to_numeric(combined_df1['Stick_fraction'][combined_df1['Max_score']== '100'])
probe_fraction_not_full_score = pd.to_numeric(combined_df1['Stick_fraction'][combined_df1['Max_score']!= '100'])
group1 = probe_fraction_full_score.to_numpy()
group2 = probe_fraction_not_full_score.to_numpy() 

df = pd.DataFrame({
    "Group": ["full score"] * len(group1) + ["not full score"] * len(group2),
    "Value": np.concatenate([group1, group2]),
    "ColorGroup": ["full score"] * len(group1) + ["not full score"] * len(group2)  # 添加颜色分组
})
plt.figure(figsize=(6, 5))
sns.violinplot(x="Group", y="Value", data=df, inner=None, color='pink', linewidth=1.5, alpha=0.2)
sns.stripplot(x="Group", y="Value", data=df, jitter=True, size=5, hue="ColorGroup", palette={"full score": "grey", "not full score": "gainsboro"}, alpha=0.8)

for group in df["Group"].unique():
    group_values = df[df["Group"] == group]["Value"]
    median = np.median(group_values)
    mean = np.mean(group_values)
    q1 = np.percentile(group_values, 25)
    q3 = np.percentile(group_values, 75)
    
    x_pos = list(df["Group"].unique()).index(group)

    plt.scatter(x_pos, median, color="red", label="Median" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, mean, color="green", label="Mean" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q1, color="purple", label="Q1 (25th Percentile)" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q3, color="orange", label="Q3 (75th Percentile)" if group == df["Group"].unique()[0] else "")

plt.legend()
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.title("One-block probe fraction for subject with full/not full score for p1 group", fontsize=16)
plt.xlabel("Group", fontsize=14)
plt.ylabel("Stick fraction", fontsize=14)
plt.show()

t_stat, p_value = ttest_ind(group1, group2, equal_var=False)  # 设置 equal_var=False 假设方差不相等
print(f"T-test: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")

# 判断显著性
if p_value < 0.05:
    print("两组之间的差异具有统计学意义（p < 0.05）。")
else:
    print("两组之间的差异不具有统计学意义（p >= 0.05）。")

In [ ]:
#用probe sequence 画不同组被试的probe frequency on trial course， 可视化stick sequence 时 将‘Probe_sequence'替换
all_probe_list_p1 = combined_df['Stick_sequence'][combined_df['Phase'] == 'p1']
all_stick_list_p1 = combined_df['Stick_sequence'][combined_df['Phase'] == 'p1'] # 用于同时画p1的probe 和 stick trial
all_probe_list_p2 = combined_df['Stick_sequence'][combined_df['Phase'] == 'p2']
all_probe_list_p2only = combined_df['Stick_sequence'][combined_df['Phase'] == 'p2only']

#将list中的字典元素转换为1，0保持为0
def convert_probe_list(probe_list):
    return [1 if isinstance(item, dict) else item for item in probe_list]

# 修改all_probe_list_p1, all_probe_list_p2, all_probe_list_p2only
all_probe_list_p1 = [convert_probe_list(probe_list) for probe_list in all_probe_list_p1]
all_stick_list_p1 = [convert_probe_list(stick_list) for stick_list in all_stick_list_p1]
all_probe_list_p2 = [convert_probe_list(probe_list) for probe_list in all_probe_list_p2]
all_probe_list_p2only = [convert_probe_list(probe_list) for probe_list in all_probe_list_p2only]

# 为每个列表补齐长度并转换为ndarray
max_length1 = max(len(lst) for lst in all_probe_list_p1)
data_visualize1 = [lst + [0] * (max_length1 - len(lst)) for lst in all_probe_list_p1]
data_array_p1 = np.array(data_visualize1)

max_length1_2 = max(len(lst) for lst in all_stick_list_p1)
data_visualize1_2 = [lst + [0] * (max_length1_2 - len(lst)) for lst in all_stick_list_p1]
data_array_p1_2 = np.array(data_visualize1_2)

max_length2 = max(len(lst) for lst in all_probe_list_p2)
data_visualize2 = [lst + [0] * (max_length2 - len(lst)) for lst in all_probe_list_p2]
data_array_p2 = np.array(data_visualize2)

max_length3 = max(len(lst) for lst in all_probe_list_p2only)
data_visualize3 = [lst + [0] * (max_length3 - len(lst)) for lst in all_probe_list_p2only]
data_array_p2only = np.array(data_visualize3)

#用p2 p2only 的数据画probe frequency + p1的线条
mean_array_p2 = np.mean(data_array_p2, axis=0)
mean_array_p2only = np.mean(data_array_p2only, axis=0)
mean_array_p1 = np.mean(data_array_p1,axis = 0)
mean_array_p1_2 = np.mean(data_array_p1_2,axis = 0) #用于画p1的probe stick 图
padded_array = np.full(50, np.nan)
padded_array[:len(mean_array_p1)] = mean_array_p1

# 计算标准误差
def compute_standard_error(data_array):
    n_trials = data_array.shape[1]
    se_array = []
    for i in range(n_trials):
        p = np.mean(data_array[:, i])  # 成功概率
        n = np.sum(~np.isnan(data_array[:, i]))  # 非 NaN 的样本数
        se = np.sqrt((p * (1 - p)) / n) if n > 0 else np.nan
        se_array.append(se)
    return np.array(se_array)

# 计算标准误差
se_p2 = compute_standard_error(data_array_p2)
se_p2only = compute_standard_error(data_array_p2only)
se_p1 = compute_standard_error(data_array_p1)
se_p1_2 = compute_standard_error(data_array_p1_2)
se_p1_padded = np.full(50, np.nan)
se_p1_padded[:len(se_p1)] = se_p1

# 绘制p1p2p3 三个trace的频率图
plt.figure(figsize=(10, 6))
x = np.arange(len(mean_array_p2))  # 序列索引作为x轴
plt.plot(x, mean_array_p2, label='p1p2-p2', color='blue')
plt.fill_between(x, mean_array_p2 - se_p2, mean_array_p2 + se_p2, color='blue', alpha=0.2)#标准差
plt.plot(x, mean_array_p2only, label='p2only', color='red')
plt.fill_between(x, mean_array_p2only - se_p2only, mean_array_p2only + se_p2only, color='red', alpha=0.2)
plt.plot(x, padded_array, label = 'p1p2-p1',color = 'grey')
plt.ylim(0,1)
plt.fill_between(
    x, 
    np.nan_to_num(padded_array - se_p1_padded, nan=np.nan), 
    np.nan_to_num(padded_array + se_p1_padded, nan=np.nan), 
    color='grey', alpha=0.2
)

# 进行统计检验（t检验）
u_statistic, p_value = mannwhitneyu(data_array_p2, data_array_p2only, alternative='two-sided')

# 对于p值做检验：假设显著性水平为0.05
significant_positions = np.where(p_value < 0.05)[0]
print(f"Positions with significant differences (p < 0.05): {significant_positions}")
# 标注显著性差异的点
for pos in significant_positions[:-3]:
    plt.text(pos, mean_array_p2[pos] + 0.05, '*', ha='center', va='bottom', color='black', fontsize=12)
for pos in significant_positions[-2:]:
    plt.text(pos, mean_array_p2only[pos] + 0.05, '*', ha='center', va='bottom', color='black', fontsize=12)

# 设置图表标题和标签
plt.title('Frequency of one-block probe on trial course',fontsize = 16)
plt.xlabel('Trial number',fontsize = 14)
plt.ylabel('One-block probe frequency',fontsize = 14)
plt.legend()
plt.show()

# 绘制p1 probe stick 两个trace的频率图
plt.figure(figsize=(10, 6))
x = np.arange(len(mean_array_p1))  # 序列索引作为x轴
plt.plot(x, mean_array_p1, label='three-block probe', color='blue')
plt.fill_between(x, mean_array_p1 - se_p1, mean_array_p1 + se_p1, color='blue', alpha=0.2)#标准差
plt.plot(x, mean_array_p1_2, label='one-block probe', color='red')
plt.fill_between(x, mean_array_p1_2 - se_p1_2, mean_array_p1_2 + se_p1_2, color='red', alpha=0.2)


# 进行统计检验（t检验）
u_statistic, p_value = mannwhitneyu(data_array_p1, data_array_p1_2, alternative='two-sided')

# 对于p值做检验：假设显著性水平为0.05
significant_positions = np.where(p_value < 0.05)[0]
print(f"Positions with significant differences (p < 0.05): {significant_positions}")
# 标注显著性差异的点
for pos in significant_positions[:-7]:
    plt.text(pos, mean_array_p1[pos] + 0.05, '*', ha='center', va='bottom', color='black', fontsize=12)
for pos in significant_positions[-6:]:
    plt.text(pos, mean_array_p1_2[pos] + 0.05, '*', ha='center', va='bottom', color='black', fontsize=12)

# 设置图表标题和标签
plt.title('Frequency of three/one-block probe on trial course',fontsize = 16)
plt.xlabel('Trial number',fontsize = 14)
plt.ylabel('Probe frequency',fontsize = 14)
plt.legend()
plt.show()





In [ ]:
#从combined_df1中用probe/stick sequence 提取相应的分数，并分别画分数的分布
stick_score_list = []
probe_score_list = []
for row_idx, row in combined_df.iterrows():  
    score_seq = row['Score_sequence']  
    probe_seq = row['Probe_sequence']  
    stick_seq = row['Stick_sequence']  

    probe_position  = [index for index, item in enumerate(probe_seq) if isinstance(item, dict)]
    stick_position  = [index for index, item in enumerate(stick_seq) if isinstance(item, dict)]
    probe_score = [score_seq[i] for i in probe_position]
    stick_score = [score_seq[i] for i in stick_position]
    probe_score_list.extend(probe_score)
    stick_score_list.extend(stick_score)

plt.figure(figsize=(8, 6))
sns.histplot(stick_score_list, kde=True, bins=10, color='skyblue')
plt.title("Score Distribution in probe trial", fontsize=16)
plt.xlabel("Score", fontsize=15)
plt.ylabel("Total counts ", fontsize=15)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
# probe/stick fraction for 3 group
probe_p1 = combined_df1['Probe_fraction']
probe_p2 = combined_df2['Probe_fraction']
probe_p2only = combined_df2only['Probe_fraction']
stick_p1 = combined_df1['Stick_fraction']
stick_p2 = combined_df2['Stick_fraction']
stick_p2only = combined_df2only['Stick_fraction']

#1. probe 3 group  (画stick图时切换为stick)
group1 = stick_p1.to_numpy()
group2 = stick_p2.to_numpy()
group3 = stick_p2only.to_numpy()

df = pd.DataFrame({
    "Group": ["p1p2-p1"] * len(group1) + ["p1p2-p2"] * len(group2) + ["p2-only"] * len(group3),
    "Value": np.concatenate([group1, group2, group3]),
    "ColorGroup": ["p1p2-p1"] * len(group1) + ["p1p2-p2"] * len(group2) + ["p2-only"] * len(group3)
})

# 比较组 1 和 2
t_stat, p_value_1_2 = ttest_ind(group1, group2)
print(f'ttest between group1 and group2: p-value = {p_value_1_2}')
# 比较组 1 和 3
t_stat, p_value_1_3 = ttest_ind(group1, group3)
print(f'ttest between group1 and group3: p-value = {p_value_1_3}')
# 比较组 2 和 3
t_stat, p_value_2_3 = ttest_ind(group2, group3)
print(f'ttest between group2 and group3: p-value = {p_value_2_3}')

# 创建小提琴图
plt.figure(figsize=(10, 6))
sns.violinplot(x="Group", y="Value", data=df, inner=None, hue="ColorGroup", palette={"p1p2-p1": "#96C37D", "p1p2-p2": "#82B0D2","p2-only":"#FA7F6F"}, linewidth=1.5,alpha=0.3)
sns.stripplot(x="Group", y="Value", data=df, jitter=True, size=5, color="grey", alpha=0.6)

# 计算并绘制统计信息
for group in df["Group"].unique():
    group_values = df[df["Group"] == group]["Value"]
    median = np.median(group_values)
    mean = np.mean(group_values)
    q1 = np.percentile(group_values, 25)
    q3 = np.percentile(group_values, 75)
    
    # 获取 x 坐标
    x_pos = list(df["Group"].unique()).index(group)
    
    # 添加统计信息到图中
    plt.scatter(x_pos, median, color="red", label="Median" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, mean, color="green", label="Mean" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q1, color="purple", label="Q1 (25th Percentile)" if group == df["Group"].unique()[0] else "")
    plt.scatter(x_pos, q3, color="orange", label="Q3 (75th Percentile)" if group == df["Group"].unique()[0] else "")
    plt.text(x=x_pos, y=mean + 0.05, s=f"Mean:{mean:.2f}", color="darkgreen", ha="center", fontsize=12)

# 添加图例
plt.legend()
plt.grid(True, linestyle='--', linewidth=0.5, alpha=0.6)
plt.xticks(fontsize=15)
plt.title("Stick fraction comparation for 3 groups", fontsize=16)
plt.xlabel("Group", fontsize=16)
plt.ylabel("Fraction", fontsize=16)
plt.show()